In [1]:
import datetime
import json
import pandas as pd
import numpy as np
import spacy
import re
import math
from unidecode import unidecode
from collections import Counter
from pymongo import MongoClient
nlp = spacy.load("es_core_news_lg")

In [2]:
client = MongoClient('mongodb://localhost:27017')
database = client['Thesis']
collectionDB = database.get_collection("text")
collectionDep_2 = database.get_collection("text_dep4")

In [3]:
stop_words = pd.read_excel('../input/stop_words.xlsx')['stop_word'].to_list()

In [4]:
def preproccess_0(texto):
    texto = texto.replace('\n', ' ')
    texto = texto.replace('\t', ' ')
    texto = re.sub(' +', ' ', texto)
    minusculas = texto.lower()
    sin_tildes = unidecode(minusculas)

    # Step 1: Eliminar caracteres especiales.
    texto_sin_caracteres_especiales = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]", " ", sin_tildes)
    # Step 2: Eliminar palabras con más de 20 caracteres o con menos de 2.
    palabras = texto_sin_caracteres_especiales.split()
    palabras_filtradas = []
    for palabra in palabras:
        if len(palabra) > 1 and len(palabra) <= 20:
            palabras_filtradas.append(palabra)
    texto_filtrado = " ".join(palabras_filtradas)     
    # Step 3: Eliminar stopwords.
    palabras2 = texto_filtrado.split()
    no_stop = [word for word in palabras2 if word not in stop_words]
    texto_sin_stopwords = " ".join(no_stop)  
    return texto_sin_stopwords

In [ ]:
print("Start  -------------------------------")
print(datetime.datetime.now().time())
count = 0
for document in collectionDB.find():
    text = document["text"]
    number = document["file"]
    texto = preproccess_0(text)
    res = dict()
    res['version'] = "0.0.1"
    res['file'] = number
    res['text'] = texto
    result_object = collectionDep_2.insert_one(res)
    
print("End -------------------------------")
print(datetime.datetime.now().time())

Start  -------------------------------
10:45:48.039789


In [ ]:
data.columns